In [94]:
import kfp

from typing import NamedTuple
import kfp.components as comp
from kfp import compiler, dsl
from kfp import dsl
from kfp.components import InputPath, OutputPath
from kubernetes.client.models import V1EnvVar
import time

In [95]:
def NameBuckets(url_minio,
                 access_key,
                 secret_key,
                 timestamp
                 ) -> list:
    
    from minio import Minio

    client = Minio(
        url_minio,
        access_key = access_key,
        secret_key = secret_key,
    )

    list_buckets = client.list_buckets()
    list_bucket_names = []

    for bucket_ in list_buckets:
        list_bucket_names.append(bucket_.name)

    return list_bucket_names

def PrintObjectsinBucket(
        url_minio,
        access_key,
        secret_key,
        bucket_name,
#        output_model_path: OutputPath(str),
        timestamp
):
    
    from minio import Minio
    from prophet import Prophet
    from prophet.serialize import  model_from_json
    import os

    print(os.getcwd())

    client = Minio(
        url_minio,
        access_key=access_key,
        secret_key=secret_key,
    )

    list_objects = client.list_objects(bucket_name)

    for obj_ in list_objects:
        print(obj_._object_name)

    list_objects = client.list_objects(bucket_name)
    print("reading objects \n")
    for item in list_objects:
        print(item._object_name)
        client.fget_object(bucket_name,item._object_name,file_path = os.path.join(os.getcwd(),"test_model.json"))

    for file in os.listdir():
        print(file)

    try:
        with open(os.path.join(os.getcwd(),"test_model.json"), 'r') as fin:
            prophet_model = model_from_json(fin.read())
    except:
        print("Model not loaded")
    





In [96]:
def ObjectsNamesPipeline(
        url: str,
        access_key: str = "",
        secret_key: str = "",
        timestamp: float = time.time()
):
    buckets_op = comp.create_component_from_func(
        NameBuckets, output_component_file = "name_buckets_component.yaml",packages_to_install= ["minio"])
    
    object_print_load_op = comp.create_component_from_func(
        PrintObjectsinBucket, output_component_file= "object_print_comp.yaml", base_image= "adcarras/ren-docker-forecast:0.0.1",packages_to_install= ["minio"]
    )

    env_var = V1EnvVar(name='HOME', value='/tmp')

    buckets_task = buckets_op(
        url,
        access_key,
        secret_key,
        timestamp
    ).add_env_variable(env_var)

    with dsl.ParallelFor(buckets_task.output) as bucket_name:
        objects_task = object_print_load_op(
            url, access_key, secret_key, bucket_name, timestamp
        ).add_env_variable(env_var)



compiler.Compiler().compile(pipeline_func = ObjectsNamesPipeline, package_path ="Objects_Pipeline.yaml")
